# Estimation

Often times, we can assume we have data that follows a random variable that normally distributed, or distributed according to some other distribution that has parameters. If we do make this assumption though, then how do we find out the value of these parameters?

We saw that the normal distribution has parameters $\mu$ (mean) and $\sigma$ (standard deviation). If we have data that we think came from a normal distribution, but we don't know exactly what kind of normal distribution, then $\mu$ and $\sigma$ are unknown and for all intents and purposes they are random variables with their own PDFs. In this section we will cover how to find the PDFs of these parameters.

Finding these PDFs is very useful! Say we have a bunch of cat heart weight measurements. We think they come from a normal distribution, but we want to know what the mean and standard deviation of that normal distribution is. There won't be an oracle that will tell us exactly, so the next best thing is to find the PDF of these random variables. If we have the PDF of $\mu$ for example, we can find out things like what the most likely value of $\mu$ is, how certain we are that $\mu$ is close to that value, and probabilitiy that $\mu$ is greater than 10 grams. 

#### Bayes Rule

<font color='red'>Bayes rule</font> is a formula that helps us find the PDF of random variables in terms of the PDF of other random variables. Bayes rule is often used in the estimation of parameters. If we're interested in finding the distribution of $\mu$ given the data we saw, then Bayes rule states:

$$p(\,\mu\,|\,\mathrm{Data})=\frac{p(\,\mathrm{Data}\,|\,\mu)\,p(\mu)}{p(\mathrm{Data})} \propto p(\,\mathrm{Data}\,|\,\mu)\,p(\mu)$$

This formula is of course the same for any parameter not just $\mu$. $p(\,\mathrm{Data}\,|\,\mu)$ is called a likelihood and $p(\mu)$ is called a prior. We will discuss what these mean.

#### Likelihoods

#### Priors